In [4]:

from sentence_transformers import SentenceTransformer, util
sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

tensor([[0.6003]], device='cuda:0')

In [15]:
import csv
# 입력 파일 경로1
input_file = '/data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/meta_key_tk_223.csv'
# 출력 파일 경로1
output_file = '/data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/title_genre.csv'

In [19]:
# title과 genres를 합치는 함수
def merge_title_and_genres(title, genres):
    return f"{title}, {', '.join(eval(genres))}"

In [20]:
# 다른 인코딩 방식 시도
try:
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        # 새로운 파일에 데이터 쓰기
        with open(output_file, 'w', newline='', encoding='utf-8') as f_out:
            writer = csv.writer(f_out)

            # 헤더 쓰기
            writer.writerow(['idx', 'imdb_id', 'merged_data'])

            # 데이터 쓰기
            for idx, row in enumerate(reader):
                # title과 genres 합치기
                merged_data = merge_title_and_genres(row['title'], row['genres'])

                # idx, imdb_id, merged_data 쓰기
                writer.writerow([idx, row['imdb_id'], merged_data])

    print("데이터가 성공적으로 저장되었습니다. 새로운 파일:", output_file)

except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

데이터가 성공적으로 저장되었습니다. 새로운 파일: /data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/title_genre.csv


In [1]:
# 입력 파일 경로2
input_file = '/data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/meta_key_tk_223.csv'
# 출력 파일 경로2
output_file = '/data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/tagline.csv'
# 출력 파일 경로2
output_file2 = '/data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/overview.csv'

In [9]:
import csv
# keywords를 문자열로 반환하는 함수
def extract_keywords(keywords):
    # 문자열로 반환
    return eval(keywords)

try:
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        # 새 파일에 keywords 값 저장
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for idx, row in enumerate(reader):
                imdb_id = row['imdb_id']
                keywords = extract_keywords(row['keywords'])
                f_out.write(f"{idx},{imdb_id},\"{' '.join(keywords)}\"\n")

    print("keywords가 성공적으로 저장되었습니다. 새로운 파일:", output_file)

except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

keywords가 성공적으로 저장되었습니다. 새로운 파일: /data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/keywords.csv


In [2]:
import csv
# keywords를 문자열로 반환하는 함수
def extract_tagline_tk(tagline_tk):
    # 문자열로 반환
    return eval(tagline_tk)

try:
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        # 새 파일에 keywords 값 저장
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for idx, row in enumerate(reader):
                imdb_id = row['imdb_id']
                tagline_tk = extract_tagline_tk(row['tagline_tk'])
                f_out.write(f"{idx},{imdb_id},\"{' '.join(tagline_tk)}\"\n")

    print("tagline_tk이 성공적으로 저장되었습니다. 새로운 파일:", output_file)

except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

tagline_tk이 성공적으로 저장되었습니다. 새로운 파일: /data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/tagline.csv


In [2]:
import csv
# keywords를 문자열로 반환하는 함수
def extract_overview_tk(overview_tk):
    # 문자열로 반환
    return eval(overview_tk)

try:
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        # 새 파일에 keywords 값 저장
        with open(output_file2, 'w', encoding='utf-8') as f_out:
            for idx, row in enumerate(reader):
                imdb_id = row['imdb_id']
                overview_tk = extract_overview_tk(row['overview_tk'])
                f_out.write(f"{idx},{imdb_id},\"{' '.join(overview_tk)}\"\n")

    print("overview_tk이 성공적으로 저장되었습니다. 새로운 파일:", output_file2)

except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

overview_tk이 성공적으로 저장되었습니다. 새로운 파일: /data/ngn_2024/movie_rec_llm/Keyword_Movie_Rec/dataset/overview.csv


In [2]:
# 사용자로부터 세 개의 단어를 입력받기
word1 = input("첫 번째 단어를 입력하세요: ")
word2 = input("두 번째 단어를 입력하세요: ")
word3 = input("세 번째 단어를 입력하세요: ")

# 입력받은 단어들을 ','로 연결하여 문장으로 반환
sentence = f"\"{word1}, {word2}, {word3}\""

# 결과 출력
print("입력한 단어들로 만든 문장:", sentence)

입력한 단어들로 만든 문장: "family, love, musical"


In [7]:
import csv
# 입력 파일 경로
input_file = './dataset/keywords.csv'

# 딕셔너리 초기화
imdb_keywords_dict = {}

try:
    with open(input_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # 각 행 처리
        for row in reader:
            imdb_id = row[1]  # imdb_id는 두 번째 열에 위치
            keywords = row[2]  # keywords는 세 번째 열에 위치
            
            # 딕셔너리에 추가
            imdb_keywords_dict[imdb_id] = keywords

    # values 값을 리스트로 변환
    keywords_list = list(imdb_keywords_dict.values())

    print("딕셔너리 생성 완료:")
    print(imdb_keywords_dict)
    print("values를 리스트로 변환 완료:")
    print(keywords_list)



except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

딕셔너리 생성 완료:
{'tt0114709': 'jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life', 'tt0113277': 'robbery detective bank obsession chase shooting thief honor murder suspense heist betrayal money gang cat and mouse criminal mastermind cult film ex-con heist movie one last job loner bank job neo-noir gun fight crime epic', 'tt0112641': 'poker drug abuse 1970s overdose illegal prostitution', 'tt0114369': 'self-fulfilling prophecy detective s.w.a.t. drug dealer evisceration lust and impulsiveness rage and hate pride and vanity immoderateness insomnia investigation pension police serial killer', 'tt0114814': 'law relatives theft criminal criminal mastermind hungarian sibling', 'tt0113247': 'paris neo-nazi skinhead robbery police brutality hip-hop gun socially deprived family breakdance arabian boredom revenge suburb violence police officer unemployment capture xenophobia', 'tt0112573': 'individual scotland in love with enemy legend independence idealism revolt t

In [26]:
import csv
# 입력 파일 경로
input_tg_file = './dataset/title_genre.csv'

# 딕셔너리 초기화
imdb_tg_dict = {}

try:
    with open(input_tg_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # 각 행 처리
        for row in reader:
            imdb_id = row[1]  # imdb_id는 두 번째 열에 위치
            title_genre = row[2]  # title_genre 세 번째 열에 위치
            
            # 딕셔너리에 추가
            imdb_tg_dict[imdb_id] = title_genre

    # values 값을 리스트로 변환
    tg_list = list(imdb_tg_dict.values())

    print("tg 딕셔너리 생성 완료:")
    print(imdb_tg_dict)
    print("values를 리스트로 변환 완료:")
    print(tg_list)



except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

tg 딕셔너리 생성 완료:
{'tt0114709': 'Toy Story, Animation, Comedy, Family', 'tt0113277': 'Heat, Action, Crime, Drama, Thriller', 'tt0112641': 'Casino, Drama, Crime', 'tt0114369': 'Se7en, Crime, Mystery, Thriller', 'tt0114814': 'The Usual Suspects, Drama, Crime, Thriller', 'tt0113247': 'La Haine, Drama', 'tt0112573': 'Braveheart, Action, Drama, History, War', 'tt0075314': 'Taxi Driver, Crime, Drama', 'tt0112471': 'Before Sunrise, Drama, Romance', 'tt0076759': 'Star Wars, Adventure, Action, Science Fiction', 'tt0110413': 'Leon: The Professional, Thriller, Crime, Drama', 'tt0110912': 'Pulp Fiction, Thriller, Crime', 'tt0111161': 'The Shawshank Redemption, Drama, Crime', 'tt0109830': 'Forrest Gump, Comedy, Drama, Romance', 'tt0110357': 'The Lion King, Family, Animation, Drama', 'tt0107207': 'In the Name of the Father, Drama', 'tt0107290': 'Jurassic Park, Adventure, Science Fiction', 'tt0108052': "Schindler's List, Drama, History, War", 'tt0083658': 'Blade Runner, Science Fiction, Drama, Thriller'

In [25]:
import csv
# 입력 파일 경로
input_overview_file = './dataset/overview.csv'

# 딕셔너리 초기화
imdb_overview_dict = {}

try:
    with open(input_overview_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # 각 행 처리
        for row in reader:
            imdb_id = row[1]  # imdb_id는 두 번째 열에 위치
            overview = row[2]  # overview 세 번째 열에 위치
            
            # 딕셔너리에 추가
            imdb_overview_dict[imdb_id] = overview

    # values 값을 리스트로 변환
    overview_list = list(imdb_overview_dict.values())

    print("overview 딕셔너리 생성 완료:")
    print(imdb_overview_dict)
    print("values를 리스트로 변환 완료:")
    print(overview_list)



except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

overview 딕셔너리 생성 완료:
{'tt0114709': 'led woody andy toys live happily room andy birthday brings buzz lightyear onto scene afraid losing place andy heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences', 'tt0113277': 'obsessive master thief neil mccauley leads top-notch crew various insane heists throughout los angeles mentally unstable detective vincent hanna pursues without rest man recognizes respects ability dedication even though aware cat-and-mouse game may end violence', 'tt0112641': 'life gambling paradise las vegas dark mafia underbelly', 'tt0114369': 'two homicide detectives desperate hunt serial killer whose crimes based seven deadly sins dark haunting film takes viewers tortured remains one victim next seasoned det sommerset researches sin effort get inside killer mind novice partner mills scoffs efforts unravel case', 'tt0114814': 'held l.a. interrogation room verbal kint attempts convince feds mythic crime lord keyser soz

In [ ]:
import csv
# 입력 파일 경로
input_overview_file = './dataset/tagline.csv'

# 딕셔너리 초기화
imdb_overview_dict = {}

try:
    with open(input_overview_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # 각 행 처리
        for row in reader:
            imdb_id = row[1]  # imdb_id는 두 번째 열에 위치
            overview = row[2]  # overview 세 번째 열에 위치
            
            # 딕셔너리에 추가
            imdb_overview_dict[imdb_id] = overview

    # values 값을 리스트로 변환
    overview_list = list(imdb_overview_dict.values())

    print("overview 딕셔너리 생성 완료:")
    print(imdb_overview_dict)
    print("values를 리스트로 변환 완료:")
    print(overview_list)



except UnicodeDecodeError:
    print("utf-8로 파일을 열 수 없습니다. 다른 인코딩 방식을 시도해주세요.")

In [11]:
### keywords
from sentence_transformers import SentenceTransformer, util
# sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_input= model.encode(sentence, convert_to_tensor=True)

key_cosim_list=[]
for i in range(len(keywords_list)):
    embedding_compare = model.encode(keywords_list[i], convert_to_tensor=True)
    sentence_sim=util.pytorch_cos_sim(embedding_input, embedding_compare)
    key_cosim_list.append(sentence_sim)

# print(cos_sim_list)


In [18]:
# 상위 10개의 인덱스와 sentence_sim 값을 출력
key_top_similarities = sorted(enumerate(key_cosim_list), key=lambda x: x[1], reverse=True)[:10]

for idx, similarity in key_top_similarities:
    print(f"Index: {idx}, Similarity: {similarity}", key_cosim_list[idx], keywords_list[idx])

Index: 186, Similarity: tensor([[0.3810]], device='cuda:0') tensor([[0.3810]], device='cuda:0') romantic comedy
Index: 35, Similarity: tensor([[0.3605]], device='cuda:0') tensor([[0.3605]], device='cuda:0') witch adolescence based on novel secret identity clock dream lion tornado scarecrow musical crow music kansas classic imaginary land cowardliness sepia color
Index: 65, Similarity: tensor([[0.3526]], device='cuda:0') tensor([[0.3526]], device='cuda:0') rebel brother sister relationship emperor space battle matter of life and death spaceship death star jabba the hutt ewoks jedi space opera
Index: 61, Similarity: tensor([[0.3228]], device='cuda:0') tensor([[0.3228]], device='cuda:0') prison street gang rape adolescence sexuality beethoven parent child relationship social worker living together society dystopia controversy controversial violence futuristic society
Index: 27, Similarity: tensor([[0.3219]], device='cuda:0') tensor([[0.3219]], device='cuda:0') fan morning musical talkie p

In [14]:
### title_genre
from sentence_transformers import SentenceTransformer, util
# sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_input= model.encode(sentence, convert_to_tensor=True)

tg_cosim_list=[]
for i in range(len(tg_list)):
    embedding_compare = model.encode(tg_list[i], convert_to_tensor=True)
    sentence_sim=util.pytorch_cos_sim(embedding_input, embedding_compare)
    tg_cosim_list.append(sentence_sim)

print(tg_cosim_list)


[tensor([[0.0431]], device='cuda:0'), tensor([[0.4998]], device='cuda:0'), tensor([[0.2409]], device='cuda:0'), tensor([[0.2580]], device='cuda:0'), tensor([[0.1661]], device='cuda:0'), tensor([[0.2965]], device='cuda:0'), tensor([[0.3078]], device='cuda:0'), tensor([[0.2862]], device='cuda:0'), tensor([[0.2653]], device='cuda:0'), tensor([[0.3549]], device='cuda:0'), tensor([[0.1988]], device='cuda:0'), tensor([[0.2356]], device='cuda:0'), tensor([[0.2456]], device='cuda:0'), tensor([[0.2686]], device='cuda:0'), tensor([[0.3534]], device='cuda:0'), tensor([[0.4880]], device='cuda:0'), tensor([[0.3971]], device='cuda:0'), tensor([[0.2361]], device='cuda:0'), tensor([[0.2798]], device='cuda:0'), tensor([[0.1615]], device='cuda:0'), tensor([[0.4504]], device='cuda:0'), tensor([[0.1223]], device='cuda:0'), tensor([[0.3418]], device='cuda:0'), tensor([[0.2645]], device='cuda:0'), tensor([[0.2568]], device='cuda:0'), tensor([[0.2757]], device='cuda:0'), tensor([[0.2200]], device='cuda:0'), 

In [17]:
# 상위 10개의 인덱스와 sentence_sim 값을 출력
tg_top_similarities = sorted(enumerate(tg_cosim_list), key=lambda x: x[1], reverse=True)[:10]

for idx, similarity in tg_top_similarities:
    print(f"Index: {idx}, Similarity: {similarity}", tg_cosim_list[idx], tg_list[idx])

Index: 46, Similarity: tensor([[0.6098]], device='cuda:0') tensor([[0.6098]], device='cuda:0') The Sound of Music, Drama, Family, Music, Romance
Index: 44, Similarity: tensor([[0.5575]], device='cuda:0') tensor([[0.5575]], device='cuda:0') It's a Wonderful Life, Drama, Family, Fantasy
Index: 1, Similarity: tensor([[0.4998]], device='cuda:0') tensor([[0.4998]], device='cuda:0') Toy Story, Animation, Comedy, Family
Index: 15, Similarity: tensor([[0.4880]], device='cuda:0') tensor([[0.4880]], device='cuda:0') The Lion King, Family, Animation, Drama
Index: 102, Similarity: tensor([[0.4835]], device='cuda:0') tensor([[0.4835]], device='cuda:0') Children of Heaven, Drama, Comedy, Family
Index: 181, Similarity: tensor([[0.4815]], device='cuda:0') tensor([[0.4815]], device='cuda:0') My Father and My Son, Comedy, Drama
Index: 28, Similarity: tensor([[0.4785]], device='cuda:0') tensor([[0.4785]], device='cuda:0') Singin' in the Rain, Comedy, Music, Romance
Index: 218, Similarity: tensor([[0.4750

In [27]:
### overview
from sentence_transformers import SentenceTransformer, util
# sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_input= model.encode(sentence, convert_to_tensor=True)

overview_cosim_list=[]
for i in range(len(overview_list)):
    embedding_compare = model.encode(overview_list[i], convert_to_tensor=True)
    sentence_sim=util.pytorch_cos_sim(embedding_input, embedding_compare)
    overview_cosim_list.append(sentence_sim)

print(overview_cosim_list)


[tensor([[0.2934]], device='cuda:0'), tensor([[0.0824]], device='cuda:0'), tensor([[0.2557]], device='cuda:0'), tensor([[0.0871]], device='cuda:0'), tensor([[0.0534]], device='cuda:0'), tensor([[0.0805]], device='cuda:0'), tensor([[0.1606]], device='cuda:0'), tensor([[0.0454]], device='cuda:0'), tensor([[0.3149]], device='cuda:0'), tensor([[0.1364]], device='cuda:0'), tensor([[0.0068]], device='cuda:0'), tensor([[0.1432]], device='cuda:0'), tensor([[0.1709]], device='cuda:0'), tensor([[0.1037]], device='cuda:0'), tensor([[0.1849]], device='cuda:0'), tensor([[0.0449]], device='cuda:0'), tensor([[0.1484]], device='cuda:0'), tensor([[0.0917]], device='cuda:0'), tensor([[0.0850]], device='cuda:0'), tensor([[0.2505]], device='cuda:0'), tensor([[0.0154]], device='cuda:0'), tensor([[0.1284]], device='cuda:0'), tensor([[0.0533]], device='cuda:0'), tensor([[0.3459]], device='cuda:0'), tensor([[0.0365]], device='cuda:0'), tensor([[0.2841]], device='cuda:0'), tensor([[0.1988]], device='cuda:0'), 

In [28]:
# 상위 10개의 인덱스와 sentence_sim 값을 출력
ov_top_similarities = sorted(enumerate(overview_cosim_list), key=lambda x: x[1], reverse=True)[:10]

for idx, similarity in ov_top_similarities:
    print(f"Index: {idx}, Similarity: {similarity}", overview_cosim_list[idx], overview_list[idx])

Index: 198, Similarity: tensor([[0.4172]], device='cuda:0') tensor([[0.4172]], device='cuda:0') mother last wishes send twins jeanne simon journey middle east search tangled roots adapted wajdi mouawad acclaimed play incendies tells powerful moving tale two young adults ' voyage core deep-rooted hatred never-ending wars enduring love
Index: 45, Similarity: tensor([[0.4029]], device='cuda:0') tensor([[0.4029]], device='cuda:0') film adaptation classic rodgers hammerstein musical based nun becomes governess austrian family
Index: 23, Similarity: tensor([[0.3459]], device='cuda:0') tensor([[0.3459]], device='cuda:0') film coming-of-age story young boy named apu life small indian village early years 20th century apu born poor brahmin family bengal father harihar priest dreams becoming successful poet playwright ; earn enough mother sarbajaya keeps family going limited resources sarbajaya resents share home harihar elderly cousin : old helpless cripple indir apu sister durga always getting 

In [31]:
sim_list = [tg_cosim_list, key_cosim_list, overview_cosim_list]

sum_sim_list = [sum(lst) for lst in zip(*sim_list)]

sorted_sums = sorted(enumerate(sum_sim_list), key=lambda x: x[1], reverse=True)

In [36]:
print(len(sorted_sums))
# 결과 출력

print("각 리스트의 원소를 더한 값과 해당 인덱스의 내림차순 정렬:")
for idx, value in sorted_sums:
    print(f"Index: {idx}, Sum: {value}, Title: {tg_list[idx]}")


223
각 리스트의 원소를 더한 값과 해당 인덱스의 내림차순 정렬:
Index: 27, Sum: tensor([[0.9908]], device='cuda:0'), Title: Singin' in the Rain, Comedy, Music, Romance
Index: 35, Sum: tensor([[0.9749]], device='cuda:0'), Title: The Wizard of Oz, Adventure, Family, Fantasy
Index: 140, Sum: tensor([[0.9365]], device='cuda:0'), Title: Amelie, Comedy, Romance
Index: 218, Sum: tensor([[0.9066]], device='cuda:0'), Title: Room, Drama, Thriller
Index: 52, Sum: tensor([[0.8564]], device='cuda:0'), Title: Cinema Paradiso, Drama, Romance
Index: 72, Sum: tensor([[0.8486]], device='cuda:0'), Title: Amadeus, Drama, History, Music
Index: 70, Sum: tensor([[0.8421]], device='cuda:0'), Title: The Godfather: Part II, Drama, Crime
Index: 28, Sum: tensor([[0.8421]], device='cuda:0'), Title: Vertigo, Mystery, Romance, Thriller
Index: 33, Sum: tensor([[0.8399]], device='cuda:0'), Title: Some Like It Hot, Comedy, Romance
Index: 126, Sum: tensor([[0.8364]], device='cuda:0'), Title: The Kid, Comedy, Drama
Index: 149, Sum: tensor([[0.834